# ArionHash R1CS & Plonk Constraints

In this SageMath notebook we compute the number of R1CS and Plonk constraints for ArionHash and compare them to Anemoi, Griffin and Poseidon.

The number of constraints for ArionHash with circulant matrix is computed via
\begin{align*}
    N_\textsf{R1CS} &= r \cdot \big( \left( n - 1 \right) \cdot \left( d_{1, \text{inc}} + 2 \right) + d_{2, \text{inc}} \big) \\
    N_{\textsf{Plonk}, 2} 
    &= r \cdot \big( (n - 1) \cdot (d_{1, inc} + 6) + d_{2, inc} - 1 \big) +
    \begin{cases}
        (r + 1) \cdot n \cdot (n - 1), & n = 2, 3 \\
        (r + 1) \cdot 4 \cdot (n - 1), & n \geq 4,
    \end{cases}
    \\
    N_{\textsf{Plonk}, 3} 
    &= r \cdot \big( (n - 1) \cdot (d_{1, inc} + 4) + d_{2, inc} \big) +
    \begin{cases}
        (r + 1) \cdot n, &n = 2, 3, \\
        (r + 1) \cdot \left( n + 2 + \left\lceil \frac{n - 3}{2} \right\rceil + \left\lceil \frac{n - 4}{2} \right\rceil \right), & n \geq 4,
    \end{cases}
\end{align*}

The number of constraints for ArionHash with swap matrix is computed via
\begin{align*}
    N_\textsf{R1CS} &= r \cdot \big( \left( n - 1 \right) \cdot \left( d_{1, \text{inc}} + 2 \right) + d_{2, \text{inc}} \big) \\
    N_{\textsf{Plonk}, 2} 
    &= r \cdot \big( (n - 1) \cdot (d_{1, inc} + 6) + d_{2, inc} - 1 \big) + 
    \begin{cases}
        (r + 1) \cdot n \cdot (n - 1), & n = 2, 3 \\
        (r + 1) \cdot 3 \cdot (n - 1), & n \geq 4,
    \end{cases}
    \\
    N_{\textsf{Plonk}, 3} 
    &= r \cdot \big( (n - 1) \cdot (d_{1, inc} + 4) + d_{2, inc} \big) +
    \begin{cases}
        (r + 1) \cdot n, &n = 2, 3, \\
        (r + 1) \cdot \left( n + \left\lceil \frac{n - 3}{2} \right\rceil \right), & n \geq 4,
    \end{cases}
\end{align*}

In [1]:
def number_of_multiplications_power_perm(d):
    if d == 3:
        return 2
    elif d == 5:
        return 3
    elif d == 7:
        return 4
    else:
        print("Exponent", d, "not implemented.")
        return

In [2]:
def arion_R1CS_constraints(d_1, d_2, n, r):
    d_1_inc = number_of_multiplications_power_perm(d_1)
    if d_2 in [121, 123, 125, 161, 257]:
        d_2_inc = 9
    else:
        factors = list((d_2 - 1).factor())
        if len(factors) == 1 and factors[0][0] == 2:
            d_2_inc = factors[0][1] + 1
        else:
            print("Exponent", d_2, "not implemented.")
            return 
    return r * ((n - 1) * (d_1_inc + 2) + d_2_inc)

def affine_layer_2_wire(n, mat_type="circ"):
    if n == 3:
        return n * (n - 1)
    if mat_type == "circ":
        return 4 * (n - 1)
    elif mat_type == "swap":
        return 3 * (n - 1)
    else:
        print("Matrix type", mat_type, "not implemented.")
        return

def arion_Plonk_2_wire_constraints(d_1, d_2, n, r, mat_type="circ"):
    d_1_inc = number_of_multiplications_power_perm(d_1)
    if d_2 in [121, 123, 125, 161, 257]:
        d_2_inc = 9
    else:
        factors = list((d_2 - 1).factor())
        if len(factors) == 1 and factors[0][0] == 2:
            d_2_inc = factors[0][1] + 1
        else:
            print("Exponent", d_2, "not implemented.")
            return 
    N = r * ((n - 1) * (d_1_inc + 6) + d_2_inc - 1)
    N += (r + 1) * affine_layer_2_wire(n, mat_type=mat_type)
    return N

def affine_layer_3_wire(n, mat_type="circ"):
    if n == 3:
        return n
    if mat_type == "circ":
        return n + 2 + ceil((n - 3) / 2) + ceil((n - 4) / 2)
    elif mat_type == "swap":
        return n + ceil((n - 3) / 2)
    else:
        print("Matrix type", mat_type, "not implemented.")
        return

def arion_Plonk_3_wire_constraints(d_1, d_2, n, r, mat_type="circ"):
    d_1_inc = number_of_multiplications_power_perm(d_1)
    if d_2 in [121, 123, 125, 161, 257]:
        d_2_inc = 9
    else:
        factors = list((d_2 - 1).factor())
        if len(factors) == 1 and factors[0][0] == 2:
            d_2_inc = factors[0][1] + 1
        else:
            print("Exponent", d_2, "not implemented.")
            return 
    N = r * ((n - 1) * (d_1_inc  + 4) + d_2_inc)
    N += (r + 1) * affine_layer_3_wire(n, mat_type=mat_type)
    return N

The number of constraints for Griffin is computed via
\begin{align*}
    N_\textsf{R1CS} &= 2 \cdot r \cdot \left( d_{inc} + n - 2 \right) \\
    N_{\textsf{Plonk}, 2} 
    &= r \cdot (2 \cdot d_{inc} + 4 \cdot n - 9) +
    \begin{cases}
        (r + 1) \cdot 5, & n = 3, \\
        7 \cdot r + 8, & n = 4, \\
        23 r \cdot 24, & n = 8, \\
        (r + 1) \cdot \left( \frac{8 \cdot n}{4} + 2 \cdot n - 4 \right) - r, & n \geq 12,
    \end{cases}
    \\
    N_{\textsf{Plonk}, 3} 
    &= r \cdot (2 \cdot d_{inc} + 3 \cdot n - 6) +
    \begin{cases}
        (r + 1) \cdot 3, & n = 3, \\
        5 \cdot r + 6, & n = 4, \\
        19 \cdot r + 20, & n = 8, \\
        (r + 1) \cdot \left( \frac{6 \cdot n}{4} + 4 \cdot \left\lfloor \frac{\frac{n}{4} - 1}{2} \right\rfloor + n \right) - r, & n \geq 12.
    \end{cases}
\end{align*}

In [3]:
def griffin_R1CS_constraints(d, n, r):
    d_inc = number_of_multiplications_power_perm(d)
    return 2 * r * (d_inc + n - 2)

def griffin_Plonk_2_wire_constraints(d, n, r):
    d_inc = number_of_multiplications_power_perm(d)
    N = r * (2 * d_inc + 4 * n - 9)
    if n == 3:
        return N + (r + 1) * 5
    elif n == 4:
        return N + (r + 1) * 8 - r
    elif n == 8:
        return N + (r + 1) * 24 - r
    elif n % 4 == 0:
        return N + (r + 1) * (8 * n / 4 + 2 * n - 4) - r
    else:
        print("Branch number", n, "not possible for Griffin.")
        return

def griffin_Plonk_3_wire_constraints(d, n, r):
    d_inc = number_of_multiplications_power_perm(d)
    N = r * (2 * d_inc + 3 * n - 6)
    if n == 3:
        return N + (r + 1) * 3
    elif n == 4:
        return N + (r + 1) * 6 - r
    elif n == 8:
        return N + (r + 1) * 20 - r
    elif n % 4 == 0:
        return N + (r + 1) * (6 * n / 4 * floor((n / 4 - 1) / 2) + n) - r
    else:
        print("Branch number", n, "not possible for Griffin.")
        return

The number of constraints for Anemoi is computed via
\begin{align*}
    N_\textsf{R1CS} &= \frac{r \cdot n}{2} \cdot \left( d_{inc} + 2\right) \\
    N_{\textsf{Plonk}, 2} 
    &= \frac{r \cdot n}{2} \cdot (d_{inc} + 7) + r \cdot \frac{n^2}{2} \\
    N_{\textsf{Plonk}, 3} 
    &= \frac{r \cdot n}{2} \cdot (d_{inc} + 5) + r \cdot
    \begin{cases}
        \frac{n^2}{2}, & n = 4, \\
        n + 10, & n = 6, \\
        n + 16, & n = 8
    \end{cases}
\end{align*}

In [4]:
def anemoi_R1CS_constraints(d, n, r):
    d_inc = number_of_multiplications_power_perm(d)
    return r * n / 2 * (d_inc + 2)

def anemoi_Plonk_2_wire_constraints(d, n, r):
    d_inc = number_of_multiplications_power_perm(d)
    N = r * n / 2 * (d_inc + 7)
    return N + r * n**2 / 2

def anemoi_Plonk_3_wire_constraints(d, n, r):
    d_inc = number_of_multiplications_power_perm(d)
    N = r * n / 2 * (d_inc + 5)
    if n == 4:
        return N + r * n**2 / 2
    elif n == 6:
        return N + r * (n + 10)
    elif n == 8:
        return N + r * (n + 16)
    else:
        print("Branch number", n, "not implemented for Anemoi.")
        return

The number of constraints for Poseidon is computed via
\begin{align*}
    N_\textsf{R1CS} &= d_{inc} \cdot \left( n \cdot r_f + r_p \right) \\
    N_{\textsf{Plonk}, 2} 
    &= d_{inc} \cdot \left( n \cdot r_f + r_p \right) + (r_f + r_p) \cdot n \cdot (n - 1) \\
    N_{\textsf{Plonk}, 3} &= d_{inc} \cdot \left( n \cdot r_f + r_p \right) + (r_f + r_p) \cdot n \cdot
    \begin{cases}
        1, & n = 2,3, \\
        1 + \left\lceil \frac{n - 3}{2} \right\rceil, & n \geq 4.
    \end{cases}
\end{align*}

In [5]:
def poseidon_R1CS_constraints(d, n, r_f, r_p):
    d_inc = number_of_multiplications_power_perm(d)
    return d_inc * (n * r_f + r_p)

def poseidon_Plonk_2_wire_constraints(d, n, r_f, r_p):
    d_inc = number_of_multiplications_power_perm(d)
    return d_inc * (n * r_f + r_p) + (r_f + r_p) * n * (n - 1)

def poseidon_Plonk_3_wire_constraints(d, n, r_f, r_p):
    d_inc = number_of_multiplications_power_perm(d)
    N = d_inc * (n * r_f + r_p)
    if n == 2:
        return N + (r_f + r_p) * n
    elif n == 3:
        return N + (r_f + r_p) * n
    else:
        return N + (r_f + r_p) * n * (1 + ceil((n - 3) / 2))

The number of constraints for Poseidon2 is computed via
\begin{align*}
    N_\textsf{R1CS} &= d_{inc} \cdot \left( n \cdot r_f + r_p \right) \\
    N_{\textsf{Plonk}, 2} 
    &= d_{inc} \cdot \left( n \cdot r_f + r_p \right) + r_p \cdot (2 \cdot t - 1) + (r_f + 1) \cdot
    \begin{cases}
        5, & n = 3, \\
        8, & n = 4, \\
        24, & n = 8, \\
        \frac{8 \cdot n}{4} + 2 \cdot n - 4, & n \geq 12,
    \end{cases}
    \\
    N_{\textsf{Plonk}, 3} 
    &= d_{inc} \cdot \left( n \cdot r_f + r_p \right) + r_p \cdot 
    \begin{rcases}
        \begin{cases}
            3 & n = 3, \\
            \left( n + 1 + \left\lceil \frac{n - 3}{2} \right\rceil \right), & n \geq 4
        \end{cases}
    \end{rcases}
    + (r_f + 1) \cdot
    \begin{cases}
        3, & n = 3, \\
        6, & n = 4, \\
        20, & n = 8, \\
        \frac{6 \cdot n}{4} + 4 \cdot \left\lfloor \frac{\frac{n}{4} - 1}{2} \right\rfloor + n, & n \geq 12.
    \end{cases}
\end{align*}

In [6]:
def poseidon2_R1CS_constraints(d, n, r_f, r_p):
    d_inc = number_of_multiplications_power_perm(d)
    return d_inc * (n * r_f + r_p)

def poseidon2_Plonk_2_wire_constraints(d, n, r_f, r_p):
    d_inc = number_of_multiplications_power_perm(d)
    N = d_inc * (n * r_f + r_p)
    if n == 3:
        N += (r_f + 1) * 5
    elif n == 4:
        N += (r_f + 1) * 8
    elif n == 8:
        N += (r_f + 1) * 24
    elif n % 4 == 0:
        N += (r_f + 1) * (8 * n / 4 + 2 * n - 4)
    else:
        print("Branch number", n, "not possible for Griffin.")
        return
    N += r_p * (2 * n - 1)
    return N

def poseidon2_Plonk_3_wire_constraints(d, n, r_f, r_p):
    d_inc = number_of_multiplications_power_perm(d)
    N = d_inc * (n * r_f + r_p)
    if n == 3:
        N += (r_f + 1) * 3
    elif n == 4:
        N += (r_f + 1) * 6
    elif n == 8:
        N += (r_f + 1) * 20
    elif n % 4 == 0:
        N += (r_f + 1) * (6 * n / 4 * floor((n / 4 - 1) / 2) + n)
    else:
        print("Branch number", n, "not possible for Griffin.")
        return
    if n == 3:
        N += r_p * n
    else:
        N += r_p * (n + 1 + ceil((n - 3) / 2))
    return N

The number of constraints for Rescue-Prime is computed via
\begin{align*}
    N_\textsf{R1CS} &= 2 \cdot d_{inc} \cdot n \cdot r \\
    N_{\textsf{Plonk}, 2} 
    &= 2 \cdot n \cdot r \cdot \left( d_{inc} + n - 1 \right) \\
    N_{\textsf{Plonk}, 3} &= 2 \cdot n \cdot r \cdot d_{inc} + 2 \cdot n \cdot 
                \begin{cases}
                    n, & n = 2, 3, \\
                    1 + \left\rceil \frac{n - 3}{2} \right\rceil, & n \geq 4
                \end{cases}
\end{align*}

In [7]:
def rescue_prime_R1CS_constraints(d, n, r):
    d_inc = number_of_multiplications_power_perm(d)
    return 2 * n * r * d_inc

def rescue_prime_Plonk_2_wire_constraints(d, n, r):
    d_inc = number_of_multiplications_power_perm(d)
    return 2 * n * r * (d_inc + n - 1)

def rescue_prime_Plonk_3_wire_constraints(d, n, r):
    d_inc = number_of_multiplications_power_perm(d)
    N = 2 * n * r * d_inc
    if n == 2:
        return N + 2 * r * n
    elif n == 3:
        return N + 2 * r * n
    else:
        return N + 2 * r * n * (1 + ceil((n - 3) / 2))

## ArionHash

### Circulant Matrix

In [8]:
arion_hash_params = [
#[d_1, d_2, n, r]
 [3, 257, 3, 8],
 [5, 257, 3, 8],
 [3, 257, 4, 7],
 [5, 257, 4, 7],
 [3, 257, 5, 7],
 [5, 257, 5, 7],
 [3, 257, 6, 7],
 [5, 257, 6, 7],
 [3, 257, 8, 6],
 [5, 257, 8, 6],
]

print("ArionHash constraints")
print("d_1", "\t", 
      "d_2", "\t", 
      "n", "\t", 
      "r", "\t", 
      "N_R1CS", "\t", 
      "N_Plonk_2_wire", "\t", 
      "N_Plonk_3_wire")
for param in arion_hash_params:
    d_1 = param[0]
    d_2 = param[1]
    n = param[2]
    r = param[3]
    print(d_1, "\t",
          d_2, "\t", 
          n, "\t", 
          r, "\t", 
          arion_R1CS_constraints(d_1, d_2, n, r), "\t\t", 
          arion_Plonk_2_wire_constraints(d_1, d_2, n, r, mat_type="circ"), "\t\t\t", 
          arion_Plonk_3_wire_constraints(d_1, d_2, n, r, mat_type="circ"))

ArionHash constraints
d_1 	 d_2 	 n 	 r 	 N_R1CS 	 N_Plonk_2_wire 	 N_Plonk_3_wire
3 	 257 	 3 	 8 	 136 		 246 			 195
5 	 257 	 3 	 8 	 152 		 262 			 211
3 	 257 	 4 	 7 	 147 		 320 			 245
5 	 257 	 4 	 7 	 168 		 341 			 266
3 	 257 	 5 	 7 	 175 		 408 			 303
5 	 257 	 5 	 7 	 203 		 436 			 331
3 	 257 	 6 	 7 	 203 		 496 			 361
5 	 257 	 6 	 7 	 238 		 531 			 396
3 	 257 	 8 	 6 	 222 		 580 			 411
5 	 257 	 8 	 6 	 264 		 622 			 453


### Swap Matrix

In [9]:
arion_hash_params = [
#[d_1, d_2, n, r]
 [3, 257, 3, 8],
 [5, 257, 3, 8],
 [3, 257, 4, 7],
 [5, 257, 4, 7],
 [3, 257, 5, 7],
 [5, 257, 5, 7],
 [3, 257, 6, 7],
 [5, 257, 6, 7],
 [3, 257, 8, 6],
 [5, 257, 8, 6],
]

print("ArionHash constraints")
print("d_1", "\t", 
      "d_2", "\t", 
      "n", "\t", 
      "r", "\t", 
      "N_R1CS", "\t", 
      "N_Plonk_2_wire", "\t", 
      "N_Plonk_3_wire")
for param in arion_hash_params:
    d_1 = param[0]
    d_2 = param[1]
    n = param[2]
    r = param[3]
    print(d_1, "\t",
          d_2, "\t", 
          n, "\t", 
          r, "\t", 
          arion_R1CS_constraints(d_1, d_2, n, r), "\t\t", 
          arion_Plonk_2_wire_constraints(d_1, d_2, n, r, mat_type="swap"), "\t\t\t", 
          arion_Plonk_3_wire_constraints(d_1, d_2, n, r, mat_type="swap"))

ArionHash constraints
d_1 	 d_2 	 n 	 r 	 N_R1CS 	 N_Plonk_2_wire 	 N_Plonk_3_wire
3 	 257 	 3 	 8 	 136 		 246 			 195
5 	 257 	 3 	 8 	 152 		 262 			 211
3 	 257 	 4 	 7 	 147 		 296 			 229
5 	 257 	 4 	 7 	 168 		 317 			 250
3 	 257 	 5 	 7 	 175 		 376 			 279
5 	 257 	 5 	 7 	 203 		 404 			 307
3 	 257 	 6 	 7 	 203 		 456 			 337
5 	 257 	 6 	 7 	 238 		 491 			 372
3 	 257 	 8 	 6 	 222 		 531 			 383
5 	 257 	 8 	 6 	 264 		 573 			 425


## Anemoi

In [10]:
anemoi_params = [
#[d, n, r]
 [3, 4, 14],
 [5, 4, 14],
 [3, 6, 12],
 [5, 6, 12],
 [3, 8, 12],
 [5, 8, 12],
]

print("Anemoi constraints")
print("d", "\t", 
      "n", "\t", 
      "r", "\t", 
      "N_R1CS", "\t",
      "N_Plonk_2_wire", "\t",
      "N_Plonk_3_wire")
for param in anemoi_params:
    d = param[0]
    n = param[1]
    r = param[2]
    print(d, "\t",
          n, "\t", 
          r, "\t", 
          anemoi_R1CS_constraints(d, n, r), "\t\t",
          anemoi_Plonk_2_wire_constraints(d, n, r), "\t\t\t",
             anemoi_Plonk_3_wire_constraints(d, n, r))

Anemoi constraints
d 	 n 	 r 	 N_R1CS 	 N_Plonk_2_wire 	 N_Plonk_3_wire
3 	 4 	 14 	 112 		 364 			 308
5 	 4 	 14 	 140 		 392 			 336
3 	 6 	 12 	 144 		 540 			 444
5 	 6 	 12 	 180 		 576 			 480
3 	 8 	 12 	 192 		 816 			 624
5 	 8 	 12 	 240 		 864 			 672


## Griffin

In [11]:
griffin_params = [
#[d, n, r]
 [3, 3, 23],
 [5, 3, 17],
 [3, 4, 23],
 [5, 4, 17],
 [3, 8, 24],
 [5, 8, 18],
]

print("Griffin constraints")
print("d", "\t", 
      "n", "\t", 
      "r", "\t", 
      "N_R1CS", "\t", 
      "N_Plonk_2_wire", "\t", 
      "N_Plonk_3_wire")
for param in griffin_params:
    d = param[0]
    n = param[1]
    r = param[2]
    print(d, "\t", 
          n, "\t", 
          r, "\t", 
          griffin_R1CS_constraints(d, n, r), "\t\t",
          griffin_Plonk_2_wire_constraints(d, n, r), "\t\t\t",
          griffin_Plonk_3_wire_constraints(d, n, r))

Griffin constraints
d 	 n 	 r 	 N_R1CS 	 N_Plonk_2_wire 	 N_Plonk_3_wire
3 	 3 	 23 	 138 		 281 			 233
5 	 3 	 17 	 136 		 243 			 207
3 	 4 	 23 	 184 		 422 			 351
5 	 4 	 17 	 170 		 348 			 295
3 	 8 	 24 	 384 		 1224 			 1004
5 	 8 	 18 	 324 		 960 			 794


## Poseidon

In [12]:
poseidon_params = [
#[d, n, r_f, r_p]
 [3, 3, 8, 84],
 [5, 3, 8, 56],
 [3, 4, 8, 84],
 [5, 4, 8, 56],
 [3, 5, 8, 84],
 [5, 5, 8, 56],
 [3, 6, 8, 84],
 [5, 6, 8, 56],
 [3, 8, 8, 84],
 [5, 8, 8, 56],
]

print("Poseidon constraints")
print("d", "\t", 
      "n", "\t", 
      "r_f", "\t", 
      "r_p", "\t", 
      "N_R1CS", "\t",
      "N_Plonk_2_wire", "\t", 
      "N_Plonk_3_wire")
for param in poseidon_params:
    d = param[0]
    n = param[1]
    r_f = param[2]
    r_p = param[3]
    print(d, "\t", 
          n, "\t", 
          r_f, "\t", 
          r_p, "\t", 
          poseidon_R1CS_constraints(d, n, r_f, r_p), "\t\t",
          poseidon_Plonk_2_wire_constraints(d, n, r_f, r_p), "\t\t\t",
          poseidon_Plonk_3_wire_constraints(d, n, r_f, r_p))

Poseidon constraints
d 	 n 	 r_f 	 r_p 	 N_R1CS 	 N_Plonk_2_wire 	 N_Plonk_3_wire
3 	 3 	 8 	 84 	 216 		 768 			 492
5 	 3 	 8 	 56 	 240 		 624 			 432
3 	 4 	 8 	 84 	 232 		 1336 			 968
5 	 4 	 8 	 56 	 264 		 1032 			 776
3 	 5 	 8 	 84 	 248 		 2088 			 1168
5 	 5 	 8 	 56 	 288 		 1568 			 928
3 	 6 	 8 	 84 	 264 		 3024 			 1920
5 	 6 	 8 	 56 	 312 		 2232 			 1464
3 	 8 	 8 	 84 	 296 		 5448 			 3240
5 	 8 	 8 	 56 	 360 		 3944 			 2408


## Poseidon2

In [13]:
poseidon_params = [
#[d, n, r_f, r_p]
    [7, 12, 8, 22],
 [3, 3, 8, 84],
 [5, 3, 8, 56],
 [3, 4, 8, 84],
 [5, 4, 8, 56],
 [3, 8, 8, 84],
 [5, 8, 8, 56],
]

print("Poseidon constraints")
print("d", "\t", 
      "n", "\t", 
      "r_f", "\t", 
      "r_p", "\t", 
      "N_R1CS", "\t",
      "N_Plonk_2_wire", "\t", 
      "N_Plonk_3_wire")
for param in poseidon_params:
    d = param[0]
    n = param[1]
    r_f = param[2]
    r_p = param[3]
    print(d, "\t", 
          n, "\t", 
          r_f, "\t", 
          r_p, "\t", 
          poseidon2_R1CS_constraints(d, n, r_f, r_p), "\t\t",
          poseidon2_Plonk_2_wire_constraints(d, n, r_f, r_p), "\t\t\t",
          poseidon2_Plonk_3_wire_constraints(d, n, r_f, r_p))

Poseidon constraints
d 	 n 	 r_f 	 r_p 	 N_R1CS 	 N_Plonk_2_wire 	 N_Plonk_3_wire
7 	 12 	 8 	 22 	 472 		 1374 			 1138
3 	 3 	 8 	 84 	 216 		 681 			 495
5 	 3 	 8 	 56 	 240 		 565 			 435
3 	 4 	 8 	 84 	 232 		 892 			 790
5 	 4 	 8 	 56 	 264 		 728 			 654
3 	 8 	 8 	 84 	 296 		 1772 			 1484
5 	 8 	 8 	 56 	 360 		 1416 			 1212


## Rescue-Prime

In [14]:
rescue_params = [
#[d, n, r]
 [3, 3, 18],
 [5, 3, 14],
 [3, 4, 14],
 [5, 4, 11],
 [3, 5, 12],
 [5, 5, 9],
 [3, 6, 8],
 [5, 6, 8],
 [3, 8, 8],
 [5, 8, 8],
]

print("Rescue-Prime constraints")
print("d", "\t", 
      "n", "\t", 
      "r", "\t",
      "N_R1CS", "\t",
      "N_Plonk_2_wire", "\t", 
      "N_Plonk_3_wire")
for param in rescue_params:
    d = param[0]
    n = param[1]
    r = param[2]
    print(d, "\t", 
          n, "\t", 
          r, "\t",
          rescue_prime_R1CS_constraints(d, n, r), "\t\t",
          rescue_prime_Plonk_2_wire_constraints(d, n, r), "\t\t\t",
          rescue_prime_Plonk_3_wire_constraints(d, n, r))

Rescue-Prime constraints
d 	 n 	 r 	 N_R1CS 	 N_Plonk_2_wire 	 N_Plonk_3_wire
3 	 3 	 18 	 216 		 432 			 324
5 	 3 	 14 	 252 		 420 			 336
3 	 4 	 14 	 224 		 560 			 448
5 	 4 	 11 	 264 		 528 			 440
3 	 5 	 12 	 240 		 720 			 480
5 	 5 	 9 	 270 		 630 			 450
3 	 6 	 8 	 192 		 672 			 480
5 	 6 	 8 	 288 		 768 			 576
3 	 8 	 8 	 256 		 1152 			 768
5 	 8 	 8 	 384 		 1280 			 896
